In [1]:
%%writefile register_training_data.py
import argparse
from azureml.core.run import Run
from azureml.core import Dataset, Datastore, Workspace
from azureml.core.runconfig import RunConfiguration
from azureml.core import Environment, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

def register_training_dataset(ds_str:str, veg_type:str, train_datastore_name='vdsmtrain',
                             ds_register_name=None, ws=None):
    """Register training datasets after data fusion step.
    Input param ds_str: string for path of the training data folder 
    Input param veg_type: string for veg_type, choices 'pv', 'npv', 'bs'
    
    Optional param training_datastore_name:, str, default = 'vdsmtrain'
            if not exist, will register with our custom blob using keyvault
    Optional param ds_register_name: dataset register name, default is 'training_'+veg_type+'_folder'
    Optional param ws:  AML workspace, if not specified, will get from default config
   
    Example:
        veg_type = 'pv'
        ds_str = f'traininginput_{veg_type}_v2/**'
        register_training_dataset(ds_str, veg_type)
    """
    run = Run.get_context()

    if ds_str is None:
        ds_str = 'traininginput_pv_v0/**'


    if ws is None:
       ws = Workspace.from_config()

    if ds_register_name is None:
       ds_register_name = 'training_'+veg_type+'_folder' 


    # implement keyvault in registering custom datastore
    if train_datastore_name in ws.datastores:
        train_datastore = Datastore.get(ws, train_datastore_name)
    else:
        train_datastore_storage_name = "asartpacedevstvdsm001"
        containername = "data"
        keyvault = ws.get_default_keyvault()
        access_key = keyvault.get_secret("asartpacedevstvdsm001-key")
        access_str = keyvault.get_secret("asartpacedevstvdsm001-connstr")
        train_datastore = Datastore.register_azure_blob_container(
                            workspace=ws,
                            datastore_name=train_datastore_name,
                            account_name=train_datastore_storage_name,
                            account_key=access_key,
                            container_name=containername
                            )


    train_dataset_custom = Dataset.Tabular.from_delimited_files(path=(train_datastore, ds_str), validate=False)
    train_dataset_custom = train_dataset_custom.register(
            ws,
            name=ds_register_name,
            tags={"purpose": "training input", "format": "csv"},
            create_new_version=True,
        ).as_named_input(ds_register_name)


def main():
    parser = argparse.ArgumentParser("Register Training Data")
    parser.add_argument("--blob_dir_name", type=str, dest='ds_str', help='input training data blob dir name')
    parser.add_argument("--veg_type", type=int, dest='veg_type', help='input veg type name')

    ### optional
    parser.add_argument("--training_data_store", type=str, dest='train_datastore_name', help='training_datastore name', default="vdsmtrain")
    parser.add_argument("--ds_register_name", type=str, dest='ds_register_name', help='register data name', default=None)
    parser.add_argument("--workspace", type=str, dest='ws', help='AML workspace', default=None)

    # args = parser.parse_args()
    args, _ = parser.parse_known_args()
    ds_str = args.ds_str
    veg_type = args.veg_type
    ds_register_name = args.ds_register_name
    ws = args.ws
    register_training_dataset(ds_str, veg_type)

if __name__ == '__main__':
    main()

Writing register_training_data.py


In [2]:
# %%writefile infer_fuse.py
# A simple pipeline structure that can run different steps in a compute cluster
import azure.core
from azureml.core import Workspace, Datastore, Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline
from azureml.core.runconfig import RunConfiguration
from azureml.widgets import RunDetails
from azureml.data import OutputFileDatasetConfig
import json

ws = Workspace.from_config()

blob_store = Datastore(ws, "workspaceblobstore")    # workspaceblobstore is the default

# Create Experiment
experiment_name = 'train_fusion'
exp = Experiment(ws, experiment_name)

# Create Compute Target
aml_compute_target = 'vdsm-cluster-l80'  # ALL SMALL LETTER, NO underscore, 16ws long only.
aml_compute = AmlCompute(ws, aml_compute_target)

# Use an existing env which was registered to the ws
# env_name = "vdsm_all" 
# rebuild_env = False   # True if you want to rebuild this env
# if rebuild_env:
#     env = Environment.from_conda_specification(name=env_name, file_path="vdsm_all.yml")
#     env.register(workspace=ws)
# else:
#     envs = Environment.list(workspace=ws)
#     if env_name in envs:
#         # Use an existing environment
#         env = Environment.get(workspace=ws, name=env_name)
#     else:
#         # Create a new env using yml
#         env = Environment.from_conda_specification(name=env_name, file_path="vdsm_all.yml")
#         # Register the environment 
#         env.register(workspace=ws)

env = Environment.get(workspace=ws,name="vdsm_all",version="1")


In [18]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.pipeline.steps import PythonScriptStep, ParallelRunStep, ParallelRunConfig
from azureml.pipeline.core.pipeline_output_dataset import PipelineOutputTabularDataset
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.core.graph import PipelineParameter

pipeline_run_config = ParallelRunConfig(environment=env,
                                        entry_script="train_fusion.py",
                                        error_threshold=-1,
                                        output_action="summary_only",
                                        compute_target=aml_compute,
                                        process_count_per_node=30,
                                        # process_count_per_node=5,
                                        run_invocation_timeout=10800,
                                        # node_count=1)
                                        node_count=3)

random_n_pipeline_param = PipelineParameter(name="random_n", default_value="100")
start_month_pipeline_param = PipelineParameter(name="start_month", default_value="2015-11")
end_month_pipeline_param = PipelineParameter(name="end_month", default_value="2021-07")
pv_pipeline_param = PipelineParameter(name="pv", default_value=False)
npv_pipeline_param = PipelineParameter(name="npv", default_value=False)

ds_list = []
for i in range(1,249,2):
    ds_list.append(DatasetConsumptionConfig(name="row_{}_{}".format(i, i+1), dataset=ws.datasets.get("row_{}_{}".format(i, i+1))))

ds_list.append(DatasetConsumptionConfig(name="row_249", dataset=ws.datasets.get("row_249")))

dataprep_step = ParallelRunStep(
    name="fusion_for_training",
    inputs=ds_list,
    arguments=[
        "--random_n", random_n_pipeline_param,
        "--PV", pv_pipeline_param,
        "--NPV", npv_pipeline_param,
        "--start_month", start_month_pipeline_param,
        "--end_month", end_month_pipeline_param
    ],
    parallel_run_config=pipeline_run_config)

# ## Veg type
# veg_type = 'pv'

# ### The blob dir name in the custom container data/
# ds_str = f'training/traininginput_{veg_type}_v2/**'

# #  AML workspace, if not specified, will get from default config
# ws = None      # if None, it's going to use Workspace.from_config()

# register_training_dataset(ds_str, veg_type)
# register_step = PythonScriptStep(
#     name="register training data",
#     script_name='register_training_data.py',
#     arguments=[
#         "--blob_dir_name",
#         ds_str,
#         "--veg_type",
#         veg_type,
#         "--workspace",
#         ws,
#         ], 
# )

steps = [dataprep_step]

pipeline = Pipeline(workspace=ws, steps=steps)
pipeline_run = exp.submit(pipeline)
print('VDSM training data fusion submitted for execution')
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/pipeline/core/_parallel_run_step_base.py:585: UserWarning: 
ParallelRunStep requires azureml-dataset-runtime[fuse,pandas] for tabular dataset.
Please add relevant package in CondaDependencies.
  UserWarning,


Created step fusion_for_training [2afcdd02][a0528346-6c47-4047-a520-f46cf5c1ba70], (This step will run and generate new outputs)
Submitted PipelineRun e898ef82-b419-4941-bff8-e8e33a9bc591
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/e898ef82-b419-4941-bff8-e8e33a9bc591?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
VDSM training data fusion submitted for execution
PipelineRunId: e898ef82-b419-4941-bff8-e8e33a9bc591
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/e898ef82-b419-4941-bff8-e8e33a9bc591?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
PipelineRun Status: NotStarted


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRun Status: Running


StepRunId: fc705f90-7f96-487f-a3cb-d9171a253039
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/fc705f90-7f96-487f-a3cb-d9171a253039?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
StepRun( fusion_for_training ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_68a9de183816686ee4a041cf7ba543cf9eae1ceace500a8412e80a59993747a4_d.txt
2022-02-03T05:26:20Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/aml-ws-spacetech001/azureml/fc705f90-7f96-487f-a3cb-d9171a253039/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/aml-ws-spacetech001/azureml/fc705f90-7f96-487f-a3cb-d9171a253039/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=750585 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_roo

'Finished'

In [3]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.pipeline.steps import PythonScriptStep, ParallelRunStep, ParallelRunConfig
from azureml.pipeline.core.pipeline_output_dataset import PipelineOutputTabularDataset
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.core.graph import PipelineParameter

pipeline_run_config = ParallelRunConfig(environment=env,
                                        entry_script="train_fusion.py",
                                        error_threshold=-1,
                                        output_action="summary_only",
                                        compute_target=aml_compute,
                                        process_count_per_node=30,
                                        # process_count_per_node=5,
                                        run_invocation_timeout=10800,
                                        # node_count=1)
                                        node_count=3)

random_n_pipeline_param = PipelineParameter(name="random_n", default_value="100")
start_month_pipeline_param = PipelineParameter(name="start_month", default_value="2015-11")
end_month_pipeline_param = PipelineParameter(name="end_month", default_value="2021-07")
pv_pipeline_param = PipelineParameter(name="pv", default_value=False)
npv_pipeline_param = PipelineParameter(name="npv", default_value=True)

ds_list = []
for i in range(1,249,2):
    ds_list.append(DatasetConsumptionConfig(name="row_{}_{}".format(i, i+1), dataset=ws.datasets.get("row_{}_{}".format(i, i+1))))

ds_list.append(DatasetConsumptionConfig(name="row_249", dataset=ws.datasets.get("row_249")))

dataprep_step = ParallelRunStep(
    name="fusion_for_training",
    inputs=ds_list,
    arguments=[
        "--random_n", random_n_pipeline_param,
        "--PV", pv_pipeline_param,
        "--NPV", npv_pipeline_param,
        "--start_month", start_month_pipeline_param,
        "--end_month", end_month_pipeline_param
    ],
    parallel_run_config=pipeline_run_config)

# ## Veg type
# veg_type = 'pv'

# ### The blob dir name in the custom container data/
# ds_str = f'training/traininginput_{veg_type}_v2/**'

# #  AML workspace, if not specified, will get from default config
# ws = None      # if None, it's going to use Workspace.from_config()

# register_training_dataset(ds_str, veg_type)
# register_step = PythonScriptStep(
#     name="register training data",
#     script_name='register_training_data.py',
#     arguments=[
#         "--blob_dir_name",
#         ds_str,
#         "--veg_type",
#         veg_type,
#         "--workspace",
#         ws,
#         ], 
# )

steps = [dataprep_step]

pipeline = Pipeline(workspace=ws, steps=steps)
pipeline_run = exp.submit(pipeline)
print('VDSM training data fusion submitted for execution')
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/pipeline/core/_parallel_run_step_base.py:585: UserWarning: 
ParallelRunStep requires azureml-dataset-runtime[fuse,pandas] for tabular dataset.
Please add relevant package in CondaDependencies.
  UserWarning,


Created step fusion_for_training [84858e07][36482a6b-e854-4966-a3c5-9e0f8443b704], (This step will run and generate new outputs)
Submitted PipelineRun fb530612-fa63-4e5f-9c1d-4531a1ff93fa
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/fb530612-fa63-4e5f-9c1d-4531a1ff93fa?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
VDSM training data fusion submitted for execution
PipelineRunId: fb530612-fa63-4e5f-9c1d-4531a1ff93fa
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/fb530612-fa63-4e5f-9c1d-4531a1ff93fa?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
PipelineRun Status: NotStarted


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRun Status: Running


StepRunId: 7cc1eb6c-b94d-4a22-a643-8123d4ccd18f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/7cc1eb6c-b94d-4a22-a643-8123d4ccd18f?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
StepRun( fusion_for_training ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_9850ea6757e075e7dfd42780681b825feaa9d6e9d1dc254aadc54ed81e8eabc3_d.txt
2022-02-03T09:12:30Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/aml-ws-spacetech001/azureml/7cc1eb6c-b94d-4a22-a643-8123d4ccd18f/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/aml-ws-spacetech001/azureml/7cc1eb6c-b94d-4a22-a643-8123d4ccd18f/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=750593 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_roo

'Finished'

In [11]:
from register_training_data import register_training_dataset


# ## Veg type
veg_type = "npv"
# ### The blob dir name in the custom container data/
ds_str = f'training/traininginput_{veg_type}_v2/**'
register_training_dataset(veg_type=veg_type, ds_str=ds_str)

In [7]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.pipeline.steps import PythonScriptStep, ParallelRunStep, ParallelRunConfig
from azureml.pipeline.core.pipeline_output_dataset import PipelineOutputTabularDataset
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.core.graph import PipelineParameter

pipeline_run_config = ParallelRunConfig(environment=env,
                                        entry_script="train_fusion.py",
                                        error_threshold=-1,
                                        output_action="summary_only",
                                        compute_target=aml_compute,
                                        process_count_per_node=30,
                                        # process_count_per_node=5,
                                        run_invocation_timeout=10800,
                                        # node_count=1)
                                        node_count=3)

random_n_pipeline_param = PipelineParameter(name="random_n", default_value="100")
start_month_pipeline_param = PipelineParameter(name="start_month", default_value="2015-11")
end_month_pipeline_param = PipelineParameter(name="end_month", default_value="2021-07")
pv_pipeline_param = PipelineParameter(name="pv", default_value=False)
npv_pipeline_param = PipelineParameter(name="npv", default_value=False)

ds_list = []
for i in range(1,249,2):
    ds_list.append(DatasetConsumptionConfig(name="row_{}_{}".format(i, i+1), dataset=ws.datasets.get("row_{}_{}".format(i, i+1))))

ds_list.append(DatasetConsumptionConfig(name="row_249", dataset=ws.datasets.get("row_249")))

dataprep_step = ParallelRunStep(
    name="fusion_for_training",
    inputs=ds_list,
    arguments=[
        "--random_n", random_n_pipeline_param,
        "--PV", pv_pipeline_param,
        "--NPV", npv_pipeline_param,
        "--start_month", start_month_pipeline_param,
        "--end_month", end_month_pipeline_param
    ],
    parallel_run_config=pipeline_run_config)

script_run_config = RunConfiguration()
script_run_config.target = aml_compute
script_run_config.environment = env

## Veg type
# veg_type = 'pv'
veg_type = PipelineParameter(name="veg_type", default_value="pv")

### The blob dir name in the custom container data/
# ds_str = f'training/traininginput_{veg_type}_v2/**'
ds_str = PipelineParameter(name = "ds_str", default_value=f'training/traininginput_{veg_type}_v2/**')

#  AML workspace, if not specified, will get from default config
# ws = ws      # if None, it's going to use Workspace.from_config()

register_step1 = PythonScriptStep(
    name="register training data",
    script_name='register_training_data.py',
    arguments=[
        "--blob_dir_name",
        ds_str,
        "--veg_type",
        veg_type,
        ], 
    runconfig = script_run_config,
)


## Veg type
veg_type="npv"
# veg_type = PipelineParameter(name="veg_type", default_value="npv")

### The blob dir name in the custom container data/
# ds_str = PipelineParameter(name = "ds_str", default_value=f'training/traininginput_{veg_type}_v2/**')
ds_str = PipelineParameter(name = "ds_str", default_value=f'training/traininginput_{veg_type}_v2/**')


register_step2 = PythonScriptStep(
    name="register training data",
    script_name='register_training_data.py',
    arguments=[
        "--blob_dir_name",
        ds_str,
        "--veg_type",
        veg_type,
        ], 
    runconfig = script_run_config,
)

## Veg type
veg_type = 'bs'
# veg_type = PipelineParameter(name="veg_type", default_value="bs")

### The blob dir name in the custom container data/
# ds_str = PipelineParameter(name = "ds_str", default_value=f'training/traininginput_{veg_type}_v2/**')


register_step3 = PythonScriptStep(
    name="register training data",
    script_name='register_training_data.py',
    arguments=[
        "--blob_dir_name",
        ds_str,
        "--veg_type",
        veg_type,
        ], 
    runconfig = script_run_config,
)

trigger_step = PythonScriptStep(
    name="trigger training pipeline",
    script_name='trigger_training.py',
    runconfig = script_run_config,
)

steps = [dataprep_step, register_step1, register_step2, register_step3, trigger_step]
# steps = [dataprep_step]

pipeline = Pipeline(workspace=ws, steps=steps)
pipeline_run = exp.submit(pipeline)
print('VDSM training data fusion submitted for execution')
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/pipeline/core/_parallel_run_step_base.py:585: UserWarning: 
ParallelRunStep requires azureml-dataset-runtime[fuse,pandas] for tabular dataset.
Please add relevant package in CondaDependencies.
  UserWarning,


Created step fusion_for_training [47cbfbba][d37d182d-2036-42df-a9ed-4e5b987fb067], (This step will run and generate new outputs)
Submitted PipelineRun 29b0f5e7-74eb-4d65-86eb-f1cbfdde8c25
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/29b0f5e7-74eb-4d65-86eb-f1cbfdde8c25?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
VDSM training data fusion submitted for execution
PipelineRunId: 29b0f5e7-74eb-4d65-86eb-f1cbfdde8c25
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/29b0f5e7-74eb-4d65-86eb-f1cbfdde8c25?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
PipelineRun Status: NotStarted


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

In [ ]:
# ## Veg type
veg_type = "pv"
# ### The blob dir name in the custom container data/
ds_str = f'training/traininginput_{veg_type}_v2/**'
register_training_dataset(veg_type=veg_type, ds_str=ds_str)

# ## Veg type
veg_type = "bs"
# ### The blob dir name in the custom container data/
ds_str = f'training/traininginput_{veg_type}_v2/**'
register_training_dataset(veg_type=veg_type, ds_str=ds_str)

In [ ]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.pipeline.steps import PythonScriptStep, ParallelRunStep, ParallelRunConfig
from azureml.pipeline.core.pipeline_output_dataset import PipelineOutputTabularDataset
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.core.graph import PipelineParameter

pipeline_run_config = ParallelRunConfig(environment=env,
                                        entry_script="train_fusion.py",
                                        error_threshold=-1,
                                        output_action="summary_only",
                                        compute_target=aml_compute,
                                        process_count_per_node=20,
                                        # process_count_per_node=5,
                                        run_invocation_timeout=10800,
                                        # node_count=1)
                                        node_count=2)

random_n_pipeline_param = PipelineParameter(name="random_n", default_value="100")
start_month_pipeline_param = PipelineParameter(name="start_month", default_value="2015-11")
end_month_pipeline_param = PipelineParameter(name="end_month", default_value="2021-07")
pv_pipeline_param = PipelineParameter(name="pv", default_value=False)
npv_pipeline_param = PipelineParameter(name="npv", default_value=False)

ds_list = []
for i in range(1,249,2):
    ds_list.append(DatasetConsumptionConfig(name="row_{}_{}".format(i, i+1), dataset=ws.datasets.get("row_{}_{}".format(i, i+1))))

# for i in range(1,9,2):
#     ds_list.append(DatasetConsumptionConfig(name="row_{}_{}".format(i, i+1), dataset=ws.datasets.get("row_{}_{}".format(i, i+1))))

ds_list.append(DatasetConsumptionConfig(name="row_249", dataset=ws.datasets.get("row_249")))

dataprep_step = ParallelRunStep(
    name="fusion_for_training",
    inputs=ds_list,
    arguments=[
        "--random_n", random_n_pipeline_param,
        "--PV", pv_pipeline_param,
        "--NPV", npv_pipeline_param,
        "--start_month", start_month_pipeline_param,
        "--end_month", end_month_pipeline_param
    ],
    parallel_run_config=pipeline_run_config)

# ## Veg type
# veg_type = 'pv'

# ### The blob dir name in the custom container data/
# ds_str = f'training/traininginput_{veg_type}_v2/**'

# #  AML workspace, if not specified, will get from default config
# ws = None      # if None, it's going to use Workspace.from_config()

# register_training_dataset(ds_str, veg_type)
# register_step = PythonScriptStep(
#     name="register training data",
#     script_name='register_training_data.py',
#     arguments=[
#         "--blob_dir_name",
#         ds_str,
#         "--veg_type",
#         veg_type,
#         "--workspace",
#         ws,
#         ], 
# )

steps = [dataprep_step]

pipeline = Pipeline(workspace=ws, steps=steps)
pipeline_run = exp.submit(pipeline)
print('VDSM training data fusion submitted for execution')
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()